In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [10]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (429.01 seconds): Collecting URLs... theverge            

n_total     25985
n_unique    25921
dtype: int64

,pubs,actual_pub
https://axios.com/syria-troop-withdrawal-trump-allies-kurds-d0686854-ff4b-468e-b012-8effbae49609.html,"realclearpolitics, axios",axios
https://breitbart.com/politics/2019/10/05/nolte-anti-kavanaugh-book-written-by-new-york-times-reporters-flops,"realclearpolitics, breitbart",breitbart
https://businessinsider.co.za/automatic-braking-cars-still-hit-pedestrians-aaa-study-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/bridgestone-tyres-south-africa-santaco-sa-taxi-toyota-minibus-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/if-air-chefs-strike-how-you-can-feed-yourself-from-south-african-airports-2019-10,"businessinsider, news24",businessinsider
https://foxnews.com/opinion/democrats-michelle-obama-run-liz-peek,"fox, realclearpolitics",fox
https://foxnews.com/opinion/steve-moore-democrats-are-wrong-middle-class-incomes-surging-thanks-to-trump-policies,"fox, realclearpolitics",fox
https://foxnews.com/politics/fox-news-poll-results-september-29-october-2-2019-wisconsin,"washingtontimes, nationalreview",washingtontimes
https://hotair.com/archives/ed-morrissey/2019/10/07/ocasio-cortez-prisons-slavery-lets-abolish-something,"townhall, hotair",hotair
https://msnbc.com/the-beat-with-ari/watch/swizz-beatz-on-working-with-beyonce-jay-z-drake-and-upending-the-art-world-70266437589,"msnbc, nbcnews",msnbc


huffpost                  68
cnn                      805
investing.com            925
politico                 219
time                      18
cnbc                     162
fox                      331
bbc                      277
businessinsider          543
morningstar               87
wsj                      110
nytimes                   96
guardian                 154
reuters                 1161
washingtontimes          459
washingtonpost           128
cbs                      408
marketwatch              147
atlantic                  99
vice                      98
npr                      847
newrepublic               20
yahoo                    365
independent              708
heritage                 287
zdnet                    390
townhall                 608
abcnews                   89
hotair                    84
cbc                      298
                        ... 
metro                    245
msnbc                    111
nationalreview           738
news24        

In [11]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [12]:
select_results = crawler.select(); display(*select_results)

100.00% (21.77 seconds): Selecting URLs... cryptonews          

n_total     4532
n_unique    4532
dtype: int64

,pubs,actual_pub


huffpost                 13
cnn                      85
investing.com           205
politico                 29
time                      7
cnbc                     35
fox                     139
bbc                      42
businessinsider         202
morningstar              11
wsj                      48
nytimes                  15
guardian                 54
reuters                 384
washingtontimes          72
washingtonpost           64
cbs                      41
marketwatch              46
atlantic                 16
vice                     35
npr                      16
newrepublic               3
yahoo                   205
independent              71
heritage                  0
zdnet                    33
townhall                 37
abcnews                  43
hotair                   41
cbc                      53
                       ... 
metro                    90
msnbc                    24
nationalreview           64
news24                  142
techcrunch          

In [13]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [14]:
download_results = crawler.download(); download_results

14.94% (598.58 seconds): Downloading... usatoday            

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


46.80% (1484.92 seconds): Downloading... usatoday            

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100.00% (4192.63 seconds): Downloading... dailymail           

,downloaded,trashed
abcnews,43,0
afr,27,0
aljazeera,25,80
americanconservative,5,2
arirang,0,0
arstechnica,14,0
arynews,16,23
atlantic,16,0
axios,36,16
bbc,36,6


In [20]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

['d80494a4761ead4276e510dc4f64762e319902e0',
 'f25b1d06e58c05786ad57fe9bea9817859de3861',
 '02d668060004e25c2299c9a23445e38a9c6ec771',
 '2d3e330325b35d2c622806feac055478a4a61559',
 '2e87c621d1ffb7f28485496eb82814891ff5dd1b',
 '40cf0a5a2b87e040cd211da1855cb615702cb0ca',
 'feefa98b143ce76ef3cadf0a2764ca634f773fee',
 '0a900188f145b48aad8c70bde741aea563dba542',
 'ef7e33fa30c8be4b6d90681b383234ee4f73e51a',
 '64cc9a5e5a51520306154fda70d482b57a7420b0',
 '12e43a97340dc5a9f9270b38a9705e461456041b',
 'd71b4a23216b50deb276d3ccd8c1905939f5a9c2',
 '9a5e5f813637acd5359da986e562fa9ad9858f4b',
 '44c36bff1fe4d64fb468131f1221959fa30718d6',
 'dbf61f3dd462f9c650d0b2f42ca69717701fc7d1',
 '385dbc7ad54410ac2078055044e34c6e30cfde4b',
 'a31fe291585e194540805022e45df9ebdbabc9fc',
 '66afae226b2dc5d329fc731f7931f9f6d6f37e23']